In [46]:
import json
import os
import bs4
from bs4 import BeautifulSoup
from glob import glob
import re
import pandas as pd

In [47]:
import os

year = "2023" 
current_directory = os.getcwd() + "/data_summarization_with_abstract/" + year + "/"


In [48]:
current_directory

'/Users/mengfeilan/Documents/multitagger/data_summarization_with_abstract/2023/'

In [49]:
def load_data(file):
    with open(file, 'r') as f:
        html_doc = ''.join(f.readlines())
    return html_doc


def sentence_extract(p, full_text):
    p_text = ''
    for n in p.contents:
        if n.name == None:
            p_text += n.replace('\n', ' ')
        elif n.name in ['xref', 'sup', 'italic']:
            p_text += n.text.replace('\n', ' ')
        elif n.name in ['table-wrap', 'fig']:
            # title of figure and table
            label = n.label.text.replace('\n', ' ') if n.label else ''
            caption = n.caption.text.replace('\n', ' ') if n.caption else ''
            caption_text = label + " " + caption + ". "
            full_text += caption_text
        elif n.name == "title":
            print("title: ", n.get_text())
            p_text += n.get_text() + "\n"
            
        elif n.name == "ext-link":
            p_text += n.get_text()
        else:
            n_text = '. '.join([i.get_text() for i in n.findAll('p')])
            p_text += n_text + " "
    full_text += p_text + '\n'
    return full_text


def paragraph_extract(sec, full_text, subsection_mark):
    if sec.find('p', recursive=False):
        for p in sec.findAll('p', recursive=False):
            full_text = sentence_extract(p, full_text)
        if subsection_mark == 0:
            full_text += '\n'
#         if subsection_mark == 1:
#             full_text += '\n'
    return full_text


def iter_extract(sec, full_text, text_section, subsection_mark):
    if sec.find('sec', recursive=False):
        section_level = 0
        pre_c = len(full_text)
        if sec.title:
            full_text += sec.title.text + '\n'
#         if sec.find('fig'):
#             for fig in sec.findAll('fig', recursive=False):
#                 label = fig.label.text.replace('\n', ' ') if fig.label else ''
#                 caption = fig.caption.text.replace('\n', ' ') if fig.caption else ''
#                 caption_text = label + " " + caption + ". "
#                 full_text += caption_text + "\n"
        
        if len(sec.findAll('sec', recursive=False)) != 0:
            section_level += 1
            subsection_mark += 1
            full_text = paragraph_extract(sec, full_text, subsection_mark)

        for i in sec.findAll('sec', recursive=False):
            full_text, text_section = iter_extract(i, full_text, text_section, subsection_mark)

        after_c = len(full_text)
        sec_name = sec.title.text if sec.title else ''
        if sec_name in text_section.keys():
            if text_section[sec_name][-1]-pre_c == -1:
                text_section[sec_name][-1] = after_c-1
                text_section[sec_name][0] = section_level
            elif text_section[sec_name][-2] == pre_c and text_section[sec_name][-1] == after_c-1:
                pass
            else:
                text_section[sec_name].extend([pre_c, after_c-1,section_level])
        else:
            text_section[sec.title.text if sec.title else ''] = [pre_c, after_c-1,section_level]

        full_text += '\n'
        return full_text, text_section
    
    elif not sec.find('p', recursive=False):
        section_level = 0
        if sec:
            pre_c = len(full_text)
            full_text = paragraph_extract(sec, full_text, subsection_mark) #change here
            full_text += '\n'
            after_c = len(full_text)
            sec_name = sec.title.text if sec.title else ''
            if sec_name in text_section.keys():
                if text_section[sec_name][-1]-pre_c == -1:
                    text_section[sec_name][-1] = after_c-1
                    text_section[sec_name][0] = section_level
                else:
                    text_section[sec_name].extend([pre_c, after_c-1,section_level])
            else:
                text_section[sec.title.text if sec.title else ''] = [pre_c, after_c-1,section_level]
            return full_text, text_section
    else:
        section_level = 0
        pre_c = len(full_text)
        if sec.title:
            full_text += sec.title.text.replace('\n', ' ') + '\n'
        full_text = paragraph_extract(sec, full_text, subsection_mark)
        # full_text += ' '.join([j.get_text().replace('\n', ' ') for j in sec.findAll('p')]) + '\n'
        after_c = len(full_text)
        sec_name = sec.title.text if sec.title else ''
        if sec_name in text_section.keys():
            if text_section[sec_name][-1]-pre_c == -1:
                text_section[sec_name][-1] = after_c-1
                text_section[sec_name][0] = section_level
            else:
                text_section[sec_name].extend([pre_c, after_c-1,section_level])
        else:
            text_section[sec.title.text if sec.title else ''] = [pre_c, after_c-1,section_level]
        return full_text, text_section



In [88]:
def extract_data(file, num=None):
    subsection_mark = 0
    html_doc = load_data(file)
    soup = BeautifulSoup(html_doc, 'html.parser').findAll('article', recursive=False)[0]

#     if soup['article-type'] != 'research-article':
#         return False, False, False

    for c in soup.children:
        if c.name not in ['front', 'body', 'back']:
            c.replaceWith('')

    section_header = []
    text_section = dict()
    article_meta = soup.findAll('article-meta')[0].findAll('article-id')

    article_meta = [i.text for i in article_meta if i['pub-id-type'] == 'pmid'][0]
#     if num:
#         if article_meta in num:
#             return False, False, False

    full_text = ''
    # title section

    title = soup.findAll('title-group')
    if len(title) == 1:
        title = title[0].find('article-title').get_text(separator=' ').replace('\n', ' ')
        full_text += title+'\n'
    else:
        # print(file, ' more than one title')
        text_section['title']=[0, full_text.__len__()-1]

    # abstract section
    abstract = soup.findAll('abstract')
    abstract_content = ""
    if abstract:
        for abs in abstract:
            if abs.get('abstract-type') not in ['toc', 'graphical', 'teaser', 'author-highlights']:
                if not abs.find('title', recursive=False):
                    full_text += '\n'
                    
                abs_sec = abs.contents
                for sec in abs_sec:
                    if type(sec) is bs4.element.NavigableString:
                        full_text += sec
                        abstract_content += sec
                    elif type(sec) is bs4.element.Comment:
                        pass
                    elif type(sec) is bs4.element.Tag:
                        if sec.title and sec.p:
#                             full_text += sec.title.text
#                             full_text += "\n"
#                             full_text += sec.p.text
#                             full_text += "\n"
                            abstract_content += sec.title.text
                            abstract_content += "\n"
                            abstract_content += sec.p.text
                            abstract_content += "\n"
                        else:
#                             full_text += sec.text
#                             full_text += '\n'
                            abstract_content += sec.text
                            abstract_content += '\n'
                    else:
                        abstract_content, text_section = iter_extract(sec, "", text_section, subsection_mark)

    else:
        abstract = ''
        try:
            text_section['abstract'] = [text_section['title'][1], full_text.__len__()-1]
        except:
            return False, False, False, False

    # body section
    body = soup.findAll('body')
    if len(body) > 0:
        full_text += '\n'
        if len(body) == 1:
            body = body[0].contents
            for sec in body:
                if type(sec) is bs4.element.NavigableString:
                    full_text += sec
                else:
                    full_text, text_section = iter_extract(sec, full_text, text_section, subsection_mark)

    # back section
    back_matter_text = ""
    back = soup.findAll('back')
    if len(back) > 0:
        for sec in back[0].contents:
            #foot note
            if sec.name == "fn-group":
                foot_notes = sec.findAll("fn")
                for foot_note in foot_notes:
                    foot_note_text = '\n'.join([i.get_text() for i in foot_note.findAll('p')])  
                    back_matter_text += foot_note_text
                back_matter_text += "\n"                      
            #acknowledgement
            if sec.name == "ack":
                try:
                    sec_title = sec.find("title").get_text()
                except:
                    sec_title = ""
                back_matter_text += sec_title
                back_matter_text += "\n"
                sec_text = ' '.join([i.get_text() for i in sec.findAll('p', recursive=False)])          
                back_matter_text += sec_text
                sub_sections = sec.findAll('sec')
                for sub_sec in sub_sections:
                    if sub_sec.find("title"):
                        back_matter_text += "\n"
                        sub_sec_title = sub_sec.find("title").get_text()
                        back_matter_text += sub_sec_title
                        back_matter_text += "\n"
                        sub_sec_text = ' '.join([i.get_text() for i in sub_sec.findAll('p')])
                        back_matter_text += sub_sec_text
                        
            #notes
            if sec.name == "notes":
                sub_sections = sec.findAll('sec')
                for sub_sec in sub_sections:
                    if sub_sec.find("title"):
                        back_matter_text += "\n" 
                        sub_sec_title = sub_sec.find("title").get_text()
                        back_matter_text += sub_sec_title
                        back_matter_text += "\n"
                        sub_sec_text = ' '.join([i.get_text() for i in sub_sec.findAll('p')])
                        back_matter_text += sub_sec_text
            
            #sections
            if sec.name == "sec":
                if sec.find("title"):
                    sec_title = sec.find("title").get_text()
                    back_matter_text += sec_title
                    back_matter_text += "\n"
                    sec_text = ' '.join([i.get_text() for i in sec.findAll('p')])
                    back_matter_text += sec_text
                    
            #glossary
            if sec.name == "glossary":
                if sec.find("title"):
                    sec_title = sec.find("title").get_text()
                    back_matter_text += sec_title
                    terms = sec.findAll("def-item")
                    for term in terms:
                        back_matter_text += "\n"
                        term_abb = term.find("term").get_text()
                        term_meaning = term.find("p").get_text()
                        back_matter_text += term_abb + " " + term_meaning
                
    full_text +=back_matter_text
            
    return abstract_content, full_text, article_meta, text_section

In [89]:
def generate_brat_data(input_folder,output_folder):   
    annotation = ""
    processed_full_text = ""
    files_list = os.listdir(input_folder)
    for PMCID in files_list:
        if PMCID.startswith('.'):
            continue
        folder_path = input_folder + PMCID + "/"
        files = os.listdir(folder_path)
        for file in files:

            if ".nxml" in file:
                file_path = folder_path + file
                abstract_content, full_text, article_meta, text_section = extract_data(file_path)
                if full_text == None:
                    continue
                text_file_name_full = output_folder + str(PMCID) + "_fulltext.txt"
                text_file_name_abs = output_folder + str(PMCID) + "_abs.txt"
                textfile_full = open(text_file_name_full,"w") 
                textfile_abs = open(text_file_name_abs, "w")
                if full_text == False:
                    continue
                if abstract_content == False:
                    continue
                textfile_full.write(full_text) 
                textfile_abs.write(abstract_content)


In [90]:
input_folder = current_directory + "NXML_second/"
print(input_folder)
output_folder = current_directory + "fulltext_and_abstract/"
generate_brat_data(input_folder,output_folder)

/Users/mengfeilan/Documents/multitagger/data_summarization_with_abstract/2023/NXML_second/


AttributeError: 'NoneType' object has no attribute 'get_text'